## Setup
1. In `cd NEMO`
1. Run YAP API server `./yap api` (if you specify a port, change it in `config.py` 
1. Run NEMO API server `uvicorn api_main:app --reload --port 8090`

In [266]:
import requests

In [270]:
requests.get('http://localhost:8090/').json()

{'error': 'Please specify command'}

In [271]:
texts = [
    'גנן גידל דגן בגן\nדגן גדול גדל בגן',
    'אובאמה הכריז אתמול בעצרת האו"ם שטראמפ ליצן.',
    'מלך השערים במונדיאל 2006 היה מירוסלב קלוזה.'
]

## Run NER Model

In [272]:
%%time 
payload = { 'sentences': texts[1],
            'model_name': 'token-multi', # / 'token-single' / 'morph' 
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/run_ner_model', params=payload).json()
list(zip(res['tokenized_text'][0], res['nemo_predictions'][0]))

CPU times: user 11 ms, sys: 0 ns, total: 11 ms
Wall time: 94.4 ms


[('אובאמה', 'S-PER'),
 ('הכריז', 'O'),
 ('אתמול', 'O'),
 ('בעצרת', 'O^O'),
 ('האו"ם', 'B-ORG^E-ORG'),
 ('שטראמפ', 'O^O'),
 ('ליצן', 'O'),
 ('.', 'O')]

### multi_to_single (get token-single predictions using a token-multi model)

In [277]:
%%time
payload = { 'sentences': texts[1],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_to_single', params=payload).json()
res

CPU times: user 11.3 ms, sys: 56 µs, total: 11.4 ms
Wall time: 83.8 ms


{'tokenized_text': [['אובאמה',
   'הכריז',
   'אתמול',
   'בעצרת',
   'האו"ם',
   'שטראמפ',
   'ליצן',
   '.']],
 'nemo_multi_predictions': [['S-PER',
   'O',
   'O',
   'O^O',
   'B-ORG^E-ORG',
   'O^O',
   'O',
   'O']],
 'single_predictions': [['S-PER', 'O', 'O', 'O', 'S-ORG', 'O', 'O', 'O']]}

## MD + NER

### multi_align_hybrid

In [273]:
%%time
payload = { 'sentences': texts[2],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_align_hybrid', params=payload).json()
res

CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 165 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'nemo_multi_predictions': [['O',
   'O^O',
   'O^O',
   'O',
   'O',
   'B-PER',
   'E-PER',
   'O']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהש

In [274]:
print(res['md_lattice'])

0	1	מלך	מלך	NNT	NNT	gen=M|num=S	1
1	2	ה	ה	DEF	DEF	_	2
2	3	שערים	שער	NN	NN	gen=M|num=P	2
3	4	ב	ב	PREPOSITION	PREPOSITION	_	3
4	5	מונדיאל	מונדיאל	NNT	NNT	gen=M|num=S	3
5	6	2006	_	CD	CD	_	4
6	7	היה	היה	COP	COP	gen=M|num=S|per=3	5
7	8	מירוסלב	מירוסלב	NNP	NNP	gen=M|num=S	6
8	9	קלוזה	קלוזה	NNP	NNP	gen=F|gen=M|num=S	7
9	10	.	_	yyDOT	yyDOT	_	8




### morph_yap

In [288]:
%%time
payload = { 'sentences': texts[2],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/morph_yap', params=payload).json()
res

CPU times: user 11.8 ms, sys: 347 µs, total: 12.1 ms
Wall time: 167 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tge

### morph_hybrid

In [295]:
%%time
payload = { 'sentences': texts[2],
            #'model_name': 'token-multi',
            #'tokenized': True,
           'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid', params=payload).json()
res

CPU times: user 9.71 ms, sys: 2.58 ms, total: 12.3 ms
Wall time: 252 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'nemo_multi_predictions': [['O',
   'O^O',
   'O^O',
   'O',
   'O',
   'B-PER',
   'E-PER',
   'O']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהש

### morph_hybrid_align_tokens

In [297]:
%%time
payload = { 'sentences': texts[2],
            #'model_name': 'token-multi',
            #'tokenized': True,
           #'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid_align_tokens', params=payload).json()
res

CPU times: user 8.77 ms, sys: 485 µs, total: 9.26 ms
Wall time: 200 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'nemo_multi_predictions': [['O',
   'O^O',
   'O^O',
   'O',
   'O',
   'B-PER',
   'E-PER',
   'O']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהש